In [26]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot  as plt
import seaborn as sn
%matplotlib inline
import pandas as pd
from tabulate import tabulate
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor

In [27]:
df = pd.read_csv("/home/growlt199/Downloads/manufacturing_project/data/flight_processed_data.csv",index_col=0)

In [28]:
df

,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,price,travel_route
0,SpiceJet,SG-8709,Delhi,Evening,0,Night,Mumbai,0,2.17,1,5953,Delhi-Mumbai
1,SpiceJet,SG-8157,Delhi,Early_Morning,0,Morning,Mumbai,0,2.33,1,5953,Delhi-Mumbai
2,AirAsia,I5-764,Delhi,Early_Morning,0,Early_Morning,Mumbai,0,2.17,1,5956,Delhi-Mumbai
3,Vistara,UK-995,Delhi,Morning,0,Afternoon,Mumbai,0,2.25,1,5955,Delhi-Mumbai
4,Vistara,UK-963,Delhi,Morning,0,Morning,Mumbai,0,2.33,1,5955,Delhi-Mumbai
...,...,...,...,...,...,...,...,...,...,...,...,...
300148,Vistara,UK-822,Chennai,Morning,1,Evening,Hyderabad,1,10.08,49,69265,Chennai-Hyderabad
300149,Vistara,UK-826,Chennai,Afternoon,1,Night,Hyderabad,1,10.42,49,77105,Chennai-Hyderabad
300150,Vistara,UK-832,Chennai,Early_Morning,1,Night,Hyderabad,1,13.83,49,79099,Chennai-Hyderabad
300151,Vistara,UK-828,Chennai,Early_Morning,1,Evening,Hyderabad,1,10.00,49,81585,Chennai-Hyderabad


In [29]:
#removing colums which i dont want to use in model 
df1= df.drop(['travel_route'],axis=1)

In [30]:
from sklearn.preprocessing import LabelEncoder

# Create an instance of LabelEncoder
label_encoder = LabelEncoder()

df1['flight'] = label_encoder.fit_transform(df1['flight'])

In [31]:
# get dummy variables for multiple columns
dummy_cols = ['airline', 'departure_time','arrival_time','source_city','destination_city']
df1_dummy = pd.get_dummies(df1[dummy_cols])

# concatenate original DataFrame with dummy variable DataFrame
df1_concat = pd.concat([df1.drop(dummy_cols, axis=1), df1_dummy], axis=1)

# display result
df1_concat.head()

,flight,stops,class,duration,days_left,price,airline_AirAsia,airline_Air_India,airline_GO_FIRST,airline_Indigo,...,source_city_Delhi,source_city_Hyderabad,source_city_Kolkata,source_city_Mumbai,destination_city_Bangalore,destination_city_Chennai,destination_city_Delhi,destination_city_Hyderabad,destination_city_Kolkata,destination_city_Mumbai
0,1408,0,0,2.17,1,5953,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
1,1387,0,0,2.33,1,5953,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
2,1213,0,0,2.17,1,5956,1,0,0,0,...,1,0,0,0,0,0,0,0,0,1
3,1559,0,0,2.25,1,5955,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
4,1549,0,0,2.33,1,5955,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1


In [32]:
#create x and y where x are features for the model and y as target
X = df1_concat.drop('price',axis=1)
y = df1_concat['price']

In [33]:
def regression(x1,y1):
    import time

    start_time = time.time()


    #use sklearn train test spilt for spilt our data in parts like training data and testing data 
    from sklearn.model_selection import train_test_split

    x_train, x_test, y_train, y_test = train_test_split(x1,y1,test_size=0.3, random_state =101, shuffle=True)

    #print shape of x train,y train, x test and y test 
    print('x_train shape:-',x_train.shape)
    print('x_test shape:-',x_test.shape)
    print('y_train shape:-',y_train.shape)
    print('y_test shape:-',y_test.shape)


    #import model from sklearn
    from sklearn.ensemble import RandomForestRegressor
    print('model name :- random forest regression')

    regr = RandomForestRegressor(random_state=123)

    #training model input as x_train and y_tarin
    regr.fit(x_train,y_train)

    print('score on the testing data :-',regr.score(x_test,y_test))
    print('score on the training data :-',regr.score(x_train,y_train))

    #finding error methods 
    from sklearn import metrics

    #model is creted show give x test data as input and retuen predictd data 
    predictions = regr.predict(x_test)

    # evaluation
    data = [
        ["MAE",metrics.mean_absolute_error(y_test,predictions)],
        ["MSE",metrics.mean_squared_error(y_test,predictions)],
        ["RMSE",np.sqrt(metrics.mean_squared_error(y_test,predictions))],
        ["r2",r2_score(y_test, predictions)],
        ["explained_variance_score",metrics.explained_variance_score(y_test, predictions)],
        ["max_error",metrics.max_error(y_test, predictions)],
        ["mean_absolute_percentage_error",metrics.mean_absolute_percentage_error(y_test, predictions)],
        ["mean_squared_log_error",metrics.mean_squared_log_error(y_test, predictions)],
        ["mean_squared_log_error(squared=False)",metrics.mean_squared_log_error(y_test, predictions, squared=False)]
    ]

    # Define the headers
    headers = ["metrics", "value"]

    # Display the data in a tabular format
    table = tabulate(data, headers, tablefmt="grid")
    print(table)
    
    # Code or function to measure execution time
    end_time = time.time()
    execution_time = end_time - start_time
    print("Execution time:", execution_time, "seconds")
    
    return regr,y_test,predictions

In [34]:
regr,y_test,predictions = regression(X,y)

x_train shape:- (209180, 35)
x_test shape:- (89649, 35)
y_train shape:- (209180,)
y_test shape:- (89649,)
model name :- random forest regression
score on the testing data :- 0.9905852383648599
score on the training data :- 0.9986261696844574
+---------------------------------------+-----------------+
| metrics                               |           value |
+=======================================+=================+
| MAE                                   |   844.545       |
+---------------------------------------+-----------------+
| MSE                                   |     4.75795e+06 |
+---------------------------------------+-----------------+
| RMSE                                  |  2181.27        |
+---------------------------------------+-----------------+
| r2                                    |     0.990585    |
+---------------------------------------+-----------------+
| explained_variance_score              |     0.990588    |
+-------------------------------------

# hyper parameter tunning

In [35]:
param_grid = {
    'n_estimators': [50,100,200, 500],
    'max_depth': [None, 5, 10,20],
    'min_samples_split': [2, 5, 20],
    'min_samples_leaf': [1, 2, 10],
    'max_features': ['sqrt', 'log2']
}

In [36]:
from itertools import product


keys, values = zip(*param_grid.items())
grid = [dict(zip(keys, v)) for v in product(*values)]

print(grid)


[{'n_estimators': 50, 'max_depth': None, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'sqrt'}, {'n_estimators': 50, 'max_depth': None, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'log2'}, {'n_estimators': 50, 'max_depth': None, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 'sqrt'}, {'n_estimators': 50, 'max_depth': None, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 'log2'}, {'n_estimators': 50, 'max_depth': None, 'min_samples_split': 2, 'min_samples_leaf': 10, 'max_features': 'sqrt'}, {'n_estimators': 50, 'max_depth': None, 'min_samples_split': 2, 'min_samples_leaf': 10, 'max_features': 'log2'}, {'n_estimators': 50, 'max_depth': None, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'sqrt'}, {'n_estimators': 50, 'max_depth': None, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'log2'}, {'n_estimators': 50, 'max_depth': None, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features'

In [37]:
# grid = [{'n_estimators': 50, 'max_depth': None, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'sqrt'}, 
#         {'n_estimators': 50, 'max_depth': None, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'log2'}]

In [38]:
#print(grid)

In [39]:
import multiprocessing
import csv
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn import metrics
from sklearn.model_selection import train_test_split

filename = '/home/growlt199/Downloads/manufacturing_project/model_training/model_performance_multiprocessing_with_flight.csv'
    
with open(filename, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(['n_estimators','max_depth','min_samples_split',
                        'min_samples_leaf','max_features',
                        'mse','mae','rmse','r2',
                        'explained_variance_score', 'max_error',
                        'mean_absolute_percentage_error', 'mean_squared_log_error', 
                        'mean_squared_log_error(squared=False)',
                        'test_score','train_score']) 



def evaluate_model(hyperparameters):
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Create a Random Forest Regression model with the given hyperparameters
    model = RandomForestRegressor(
        n_estimators=hyperparameters['n_estimators'],
        max_depth=hyperparameters['max_depth'],
        min_samples_split=hyperparameters['min_samples_split'],
        min_samples_leaf=hyperparameters['min_samples_leaf'],
        max_features=hyperparameters['max_features'],
        random_state=42
    )
    
    # Fit the model to the training data
    model.fit(X_train, y_train)
    
    # Evaluate the model on the testing data
    y_pred = model.predict(X_test)
    
    # Calculate the mean squared error (MSE)
    mse = mean_squared_error(y_test, y_pred)
    
    #MAE
    mae = metrics.mean_absolute_error(y_test,y_pred)
#     print('MAE:-',metrics.mean_absolute_error(y_test,y_pred))

    #RMSE
    rmse = np.sqrt(metrics.mean_squared_error(y_test,y_pred))

    #r2
    r2 = r2_score(y_test, y_pred)
    
    #test and train accurancy
    test_score = model.score(X_test,y_test)
    train_score = model.score(X_train,y_train)
    
    explained_variance_score = metrics.explained_variance_score(y_test, y_pred)
    max_error = metrics.max_error(y_test, y_pred)
    mean_absolute_percentage_error = metrics.mean_absolute_percentage_error(y_test, y_pred)
    mean_squared_log_error = metrics.mean_squared_log_error(y_test, y_pred)
    mean_squared_log_error_squared_False = metrics.mean_squared_log_error(y_test, y_pred, squared=False)
    

    
    filename = '/home/growlt199/Downloads/manufacturing_project/model_training/model_performance_multiprocessing_with_flight.csv'
    
    with open(filename, 'a', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow([hyperparameters['n_estimators'],hyperparameters['max_depth'],
                            hyperparameters['min_samples_split'],
                            hyperparameters['min_samples_leaf'],hyperparameters['max_features'],
                            mse,mae,rmse,r2,
                            explained_variance_score, max_error,
                            mean_absolute_percentage_error, mean_squared_log_error, 
                            mean_squared_log_error_squared_False,
                            test_score,train_score]) 
    
def perform_grid_search(grid):
    pool = multiprocessing.Pool()  # Create a multiprocessing pool
    #results = []

    for hyperparameters in grid:
        result = pool.apply_async(evaluate_model(hyperparameters))
#         results.append(result)

    pool.close()
    pool.join()
    
    
#     best_hyperparameters = None
#     best_performance = float('inf')

#     for result in results:
#         performance = result.get()
#         if performance < best_performance:
#             best_performance = performance
#             best_hyperparameters = result.args[0]

#     return best_hyperparameters

In [ ]:
perform_grid_search(grid)